### 신경망의 매개변수 튜닝을 위한 Hyperopt을 활용한 예제

### 학습 내용
 * 데이터 준비 및 분할
 * 신경망 모델과 매개변수 설정
 * Hyperopt을 사용한 매개변수 탐색
 * 최적 매개변수 및 점수 출력 및 결과 분석

### 데이터 준비 및 분할

In [1]:
# ---------------------------------
# 데이터 등의 사전 준비
# 2021/04/27 시간 확인을 위한 time 관련 내용 추가
# ----------------------------------
import numpy as np
import pandas as pd
import time

# train_x는 학습 데이터, train_y는 목적 변수, test_x는 테스트 데이터
# pandas의 DataFrame, Series로 유지합니다.(numpy의 array로 유지하기도 합니다)
train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')

In [2]:
# 학습 데이터를 학습 데이터와 검증 데이터로 나누기
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [3]:
# tensorflow의 경고억제
# 23/06 수정 - 최신 버전에서 tf.compat.v1.logging 삭제하고, tf.get_logger() 함수 사용. 이에 따라 변경.

#### 변경전 기존의 코드
``` 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
```

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

### 신경망 모델과 매개변수 설정

In [7]:
# -----------------------------------
# 신경망 매개변수의 튜닝의 예
# -----------------------------------
from hyperopt import hp
from keras.callbacks import EarlyStopping

# 최신 버전에서 삭제되고 변경되어, 라이브러리  임포트 부분 일부 수정 - 23/06/16
# from keras.layers.advanced_activations import ReLU, PReLU
# from keras.layers.normalization import BatchNormalization
from keras.layers import BatchNormalization # 23/06/16 수정
from keras.layers import ReLU, PReLU  # 23/06 변경 - keras.layers.advanced_activations 최신버전에서 삭제
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler

# 기본이 되는 매개변수
base_param = {
    'input_dropout': 0.0,
    'hidden_layers': 3,
    'hidden_units': 96,
    'hidden_activation': 'relu',
    'hidden_dropout': 0.2,
    'batch_norm': 'before_act',
    'optimizer': {'type': 'adam', 'lr': 0.001},
    'batch_size': 64,
}

# 탐색할 매개변수 공간을 지정
param_space = {
    'input_dropout': hp.quniform('input_dropout', 0, 0.2, 0.05),
    'hidden_layers': hp.quniform('hidden_layers', 2, 4, 1),
    'hidden_units': hp.quniform('hidden_units', 32, 256, 32),
    'hidden_activation': hp.choice('hidden_activation', ['prelu', 'relu']),
    'hidden_dropout': hp.quniform('hidden_dropout', 0, 0.3, 0.05),
    'batch_norm': hp.choice('batch_norm', ['before_act', 'no']),
    'optimizer': hp.choice('optimizer',
                           [{'type': 'adam',
                             'lr': hp.loguniform('adam_lr', np.log(0.00001), np.log(0.01))},
                            {'type': 'sgd',
                             'lr': hp.loguniform('sgd_lr', np.log(0.00001), np.log(0.01))}]),
    'batch_size': hp.quniform('batch_size', 32, 128, 32),
}

### 다층 퍼셉트론(MLP) 클래스 구현

In [8]:
class MLP:

    def __init__(self, params):
        self.params = params
        self.scaler = None
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):

        # 매개변수
        input_dropout = self.params['input_dropout']
        hidden_layers = int(self.params['hidden_layers'])
        hidden_units = int(self.params['hidden_units'])
        hidden_activation = self.params['hidden_activation']
        hidden_dropout = self.params['hidden_dropout']
        batch_norm = self.params['batch_norm']
        optimizer_type = self.params['optimizer']['type']
        optimizer_lr = self.params['optimizer']['lr']
        batch_size = int(self.params['batch_size'])

        # 표준화
        self.scaler = StandardScaler()
        tr_x = self.scaler.fit_transform(tr_x)
        va_x = self.scaler.transform(va_x)

        self.model = Sequential()

        # 입력계층
        self.model.add(Dropout(input_dropout, input_shape=(tr_x.shape[1],)))

        # 은닉계층
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm == 'before_act':
                self.model.add(BatchNormalization())
            if hidden_activation == 'prelu':
                self.model.add(PReLU())
            elif hidden_activation == 'relu':
                self.model.add(ReLU())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))

        # 출력 계층
        self.model.add(Dense(1, activation='sigmoid'))

        # 최적화(옵티마이저)
        if optimizer_type == 'sgd':
            optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        elif optimizer_type == 'adam':
            optimizer = Adam(lr=optimizer_lr, beta_1=0.9, beta_2=0.999, decay=0.)
        else:
            raise NotImplementedError

        # 목적함수, 평가지표 등의 설정
        self.model.compile(loss='binary_crossentropy',
                           optimizer=optimizer, metrics=['accuracy'])

        # 에폭 수, 조기 종료
        # 에폭을 너무 크게 하면 작은 학습률일 때 끝나지 않을 수 있으므로 주의
        nb_epoch = 200
        patience = 20
        early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)

        # 학습의 실행
        history = self.model.fit(tr_x, tr_y,
                                 epochs=nb_epoch,
                                 batch_size=batch_size, verbose=1,
                                 validation_data=(va_x, va_y),
                                 callbacks=[early_stopping])

    def predict(self, x):
        # 예측
        x = self.scaler.transform(x)
        y_pred = self.model.predict(x)
        y_pred = y_pred.flatten()
        return y_pred

### hyperopt에 의한 매개변수 탐색

In [9]:
# -----------------------------------
# 매개변수 튜닝의 실행
# -----------------------------------
from hyperopt import fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # 매개변수 셋을 지정했을 때, 최소화해야 할 함수를 지정
    # 모델의 매개변수 탐색에서는 모델에 매개변수를 지정하여 학습예측한 경우의 점수로 함
    model = MLP(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 정보를 기록
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# hyperopt에 의한 매개변수 탐색의 실행
max_evals = 10
trials = Trials()
history = []
fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\totofriend\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1/200                                                                                                            

118/118 [==============================] - 4s 13ms/step - loss: 0.6062 - accuracy: 0.6989 - val_loss: 0.6264 - val_accuracy: 0.7276

Epoch 2/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.5549 - accuracy: 0.7560 - val_loss: 0.5385 - val_accuracy: 0.8108

Epoch 3/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.5231 - accuracy: 0.7847 - val_loss: 0.4935 - val_accuracy: 0.8140

Epoch 4/200                                                                                                            

 57/118 [=============>................] - ETA: 0s - loss: 0.5088 - accuracy: 0.7988                                  


118/118 [==============================] - 1s 9ms/step - loss: 0.5088 - accuracy: 0.7957 - val_loss: 0.4701 - val_accuracy: 0.8152

Epoch 5/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.4934 - accuracy: 0.8009 - val_loss: 0.4552 - val_accuracy: 0.8176

Epoch 6/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.4875 - accuracy: 0.8080 - val_loss: 0.4456 - val_accuracy: 0.8188

Epoch 7/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.4792 - accuracy: 0.8065 - val_loss: 0.4373 - val_accuracy: 0.8204

Epoch 8/200                                                                                                  

118/118 [==============================] - 1s 9ms/step - loss: 0.4711 - accuracy: 0.8084 - val_loss: 0.4313 - val_accuracy: 0.8212

Epoch 9/200                                                                                                            

118/118 [==============================] - 1s 9ms/step - loss: 0.4732 - accuracy: 0.8057 - val_loss: 0.4261 - val_accuracy: 0.8228

Epoch 10/200                                                                                                           

118/118 [==============================] - 1s 9ms/step - loss: 0.4648 - accuracy: 0.8125 - val_loss: 0.4217 - val_accuracy: 0.8224

Epoch 11/200                                                                                                           

101/118 [========================>.....] - ETA: 0s - loss: 0.4580 - accuracy: 0.8083                                  


...

79/79 [==============================] - 0s 4ms/step - loss: 0.3081 - accuracy: 0.8751 - val_loss: 0.3129 - val_accuracy: 0.8708

Epoch 11/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.3048 - accuracy: 0.8765 - val_loss: 0.3289 - val_accuracy: 0.8668

Epoch 12/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2958 - accuracy: 0.8780 - val_loss: 0.3401 - val_accuracy: 0.8460

Epoch 13/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2904 - accuracy: 0.8827 - val_loss: 0.3252 - val_accuracy: 0.8600

Epoch 14/200                                                                                                         

79/79 [==============================] - 0s 4ms/step - loss: 0.2835 - accuracy: 0.8868 - val_loss: 0.3138 - val_accuracy: 0.8656

Epoch 21/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2846 - accuracy: 0.8869 - val_loss: 0.3105 - val_accuracy: 0.8680

Epoch 22/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2736 - accuracy: 0.8921 - val_loss: 0.3334 - val_accuracy: 0.8616

Epoch 23/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2680 - accuracy: 0.8911 - val_loss: 0.3230 - val_accuracy: 0.8644

Epoch 24/200                                                                                                         

79/79 [==============================] - 0s 4ms/step - loss: 0.2526 - accuracy: 0.9015 - val_loss: 0.3584 - val_accuracy: 0.8500

Epoch 31/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2579 - accuracy: 0.8971 - val_loss: 0.3248 - val_accuracy: 0.8668

Epoch 32/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2578 - accuracy: 0.8967 - val_loss: 0.3319 - val_accuracy: 0.8584

Epoch 33/200                                                                                                           

79/79 [==============================] - 0s 4ms/step - loss: 0.2585 - accuracy: 0.8972 - val_loss: 0.3404 - val_accuracy: 0.8548

Epoch 34/200                                                                                                         

79/79 [==============================] - 0s 1ms/step                                                                  

params: {'batch_norm': 'no', 'batch_size': 96.0, 'hidden_activation': 'prelu', 'hidden_dropout': 0.05, 'hidden_layers': 2.0, 'hidden_units': 96.0, 'input_dropout': 0.2, 'optimizer': {'lr': 0.004716711588784886, 'type': 'adam'}}, logloss: 0.3080
100%|███████████████████████████████████████████████| 10/10 [15:01<00:00, 90.19s/trial, best loss: 0.27515965199834075]


{'adam_lr': 0.00037330699681156066,
 'batch_norm': 0,
 'batch_size': 96.0,
 'hidden_activation': 1,
 'hidden_dropout': 0.1,
 'hidden_layers': 3.0,
 'hidden_units': 96.0,
 'input_dropout': 0.05,
 'optimizer': 0}

In [10]:
# 기록한 정보에서 매개변수와 점수를 출력
# trials에서도 정보를 취득할 수 있지만 매개변수를 취득하기 어려움
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'batch_norm': 'before_act', 'batch_size': 96.0, 'hidden_activation': 'relu', 'hidden_dropout': 0.1, 'hidden_layers': 3.0, 'hidden_units': 96.0, 'input_dropout': 0.05, 'optimizer': {'lr': 0.00037330699681156066, 'type': 'adam'}}, score:0.2752
